In [47]:
from keras.layers import Activation, Dense, Input, Subtract
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_excel
import os
import pandas as pd
import seaborn as sns
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

## Load Data

In [51]:
path = "/Users/joshua/Desktop/作品集整理/LTR/txc_data2"
dirs = os.listdir( path )   
all_data = pd.DataFrame([])
for file in dirs:
   file = "/Users/joshua/Desktop/作品集整理/LTR/txc_data2/"+file
   raw_data = read_excel(file, header=0, parse_dates=[0], squeeze=True, index_col = False,skiprows = [0])
   raw_data = raw_data.iloc[1:,:]#拿掉spec row
   #還不知道要drops8哪些columns
   all_data = all_data.append(raw_data) 

## Data Preprocess

In [ ]:
# 篩選資料
# 因商業關係，特徵值用 feature no. 代替
pass_data = all_data[[ 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11']]
pass_data = pass_data.dropna()

In [ ]:
column_names = list(pass_data.columns.values) #23767 #11 rows

In [ ]:
#資料篩選
pass_data = pass_data.loc[pass_data['feature10'] != "DLD Dead"]#23737
pass_data = pass_data.loc[pass_data['feature11'] <180]#21846
pass_data = pass_data.loc[abs(pass_data['feature1']-686) <= 5]#21755
pass_data = pass_data.loc[abs(pass_data['feature2']-535) <= 5]#21431
pass_data = pass_data.loc[abs(pass_data['feature3']-439) <= 5]#21428
pass_data = pass_data.loc[abs(pass_data['feature4']-389) <= 5]#21425

In [29]:
# 型態 float --> np.float
pass_data['feature11'] = pass_data['feature11'].astype(float)
pass_data['feature3'] = pass_data['feature3'].astype(float)
pass_data['feature4'] = pass_data['feature4'].astype(float)

## Normalization

In [30]:
# 可嘗試不同的normalization
def normalize(x, y, f):
    spec_value = [686,535,439,389,52, 0]
    y_nor = y-np.mean(y) # 平移至0附近 y.mean=74.120064
    x_nor = np.copy(x)
    count = 0
    for s in spec_value:
        x_nor[:, count] -= s
        count+=1
    if(f==7):
        x_nor[:, 6] = x_nor[:, 6]/x_nor[:, 6].max(axis=0)  
    #X座標, Y座標
    if(f==8):
        x_nor[:, 6:7] = x_nor[:, 6:7]/x_nor[:, 6:7].max(axis=0)   
    #X座標, Y座標, 半徑
    if(f==9):
        x_nor[:, 6:8] = x_nor[:, 6:8]/x_nor[:, 6:8].max(axis=0)
    return x_nor, y_nor

In [31]:
fnum=8
rs = 7

if(fnum==6):
    data = pass_data[['feature11', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6']]
if(fnum==7):
    data = pass_data[['feature11', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature9']]
if(fnum==8):
    data = pass_data[['feature11', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6','feature7', 'feature8']]
if(fnum==9):
    data = pass_data[['feature11', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6','feature7', 'feature8', 'feature9']]# shuffle

In [34]:
# ---x: features(物理測量)---
x = data.drop(['feature11'],axis = 1)
x = np.asarray(x,dtype=np.float32)
#%%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=rs)
x_test_nor, y_test_nor = normalize(x_test, y_test, x.shape[1])
x_train_nor, y_train_nor = normalize(x_train, y_train, x.shape[1])

In [33]:
y = data['feature11'][:]
y = np.asarray(y,dtype=np.float32)
y = y.reshape(len(y), 1)

## Sampling

In [35]:
#%%get sample function
def randomsample(x):
    pair_num = 100000
    a = np.asarray([np.random.randint(0,x.shape[0]) for _ in range(pair_num*2)])
    first_pair = np.zeros((pair_num,x.shape[1]))
    sec_pair = np.zeros((pair_num,x.shape[1]))
    label = np.zeros((pair_num,1))
    for i in range(pair_num):
        first_pair[i,:] = x_train_nor[a[i*2]]
        sec_pair[i,:] = x_train_nor[a[i*2+1]]
        if (y_train_nor[a[i*2]]<=y_train_nor[a[i*2+1]]):
            label[i] = 1
        else:
            label[i] = 0
    return first_pair, sec_pair, label

def randomtestsample(x_test):
    pair_num = 10000
    a = np.asarray([np.random.randint(0,x_test.shape[0]) for _ in range(pair_num*2)])
    first_pair = np.zeros((pair_num,x_test.shape[1]))
    sec_pair = np.zeros((pair_num,x_test.shape[1]))
    label = np.reshape(np.zeros((pair_num,1)),(pair_num,1))
    for i in range(pair_num):
        first_pair[i,:] = x_test_nor[a[i*2]]
        sec_pair[i,:] = x_test_nor[a[i*2+1]]
        if (y_test_nor[a[i*2]]<=y_test_nor[a[i*2+1]]):
            label[i] = 1
        else:
            label[i] = 0
    return first_pair, sec_pair, label

In [36]:
t1,t2,y_test = randomtestsample(x_test)

## LTR Model

In [38]:
INPUT_DIM = x.shape[1]
# Model.
h_1 = Dense(32, activation = "relu",name = 'h1')
h_2 = Dense(16, activation = "relu",name = 'h2')
h_3 = Dense(8, activation = "relu",name = 'h3')
s = Dense(1)

In [39]:
# Relevant document score.
first_pair = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_first = h_1(first_pair)
h_2_first = h_2(h_1_first)
h_3_first = h_3(h_2_first)
first_score = s(h_3_first)

In [40]:
# Irrelevant document score.
sec_pair = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_sec = h_1(sec_pair)
h_2_sec = h_2(h_1_sec)
h_3_sec = h_3(h_2_sec)
sec_score = s(h_3_sec)

In [41]:
diff = Subtract()([first_score, sec_score])
prob = Activation("sigmoid")(diff)
model = Model(inputs = [first_pair, sec_pair], outputs = prob)
model.summary()
model.compile(optimizer = "adadelta", loss = "binary_crossentropy",metrics=["accuracy"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 h1 (Dense)                     (None, 32)           288         ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 h2 (Dense)                     (None, 16)           528         ['h1[0][0]',                 

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 16
loss = []
val_loss = []
acc = []
val_acc = []
print("rs="+str(rs)+" f="+str(fnum))
for i in range(10):
    
    print(i)
    x1,x2,y = randomsample(x_train)#first_pair, sec_pair, label
#    if i ==0:
#        weights = model.get_weights()
    
#    model.set_weights(weights)
    history = model.fit([x1, x2], y,validation_split=0.2 ,batch_size = BATCH_SIZE, epochs = NUM_EPOCHS, verbose = 0)
    weights = model.get_weights()
    loss.extend(history.history['loss'])
    val_loss.extend(history.history['val_loss'])
    acc.extend(history.history['acc'])
    val_acc.extend(history.history['val_acc'])


s = model.predict([t1,t2])
B = np.where(s > 0.5, 1, 0)
err = np.sum(abs(B-y_test))/20000

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)
#train = model.fit(x_train_nor, y_train_nor, epochs=300, validation_data=(x_test_nor,y_test_nor), batch_size=256,verbose = 0, callbacks=[early_stopping])
# #plot result
#l, vl = "%.2f" % train.history['loss'][-1], "%.2f" % train.history['val_loss'][-1]
#plt.title("< 7 features >\n rs = "+str(rs)+"\n loss="+str(l)+"  val_loss="+str(vl))
#plt.plot(train.history['loss'], label = 'loss' )
#plt.plot(train.history['val_loss'], label = 'val_loss' )
#plt.legend()
#plt.show()
a, va =  "%.3f" % acc[-1],  "%.3f" % val_acc[-1]
plt.title("< "+str(x.shape[1])+" features >\nrs = "+str(rs)+"\n acc="+str(a)+"  val_acc="+str(va))
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.legend()
plt.show()